# Liquidity pools

In [ ]:
#| default_exp pool

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from dataclasses import dataclass
from typing import Tuple, Dict, Optional, List
from sugar.token import Token
from sugar.helpers import normalize_address

In [ ]:
#| export 

@dataclass(frozen=True)
class Price:
    """Data class for Token Price

    based on:
    https://github.com/velodrome-finance/oracle/blob/main/contracts/VeloOracle.sol
    """

    token: Token
    price: float

    @property
    def pretty_price(self) -> float: return round(self.price, 5)

In [ ]:
#| export

@dataclass(frozen=True)
class Amount:
    token: Token
    amount: float
    price: "Price"

    @classmethod
    def build(cls, address: str, amount: float, tokens: Dict[str, Token], prices: Dict[str, "Price"]) -> "Amount":
        address = normalize_address(address)
        if address not in tokens or address not in prices: return None
        token = tokens[address]
        return Amount(token=token, amount=token.value_from_bigint(amount), price=prices[address])

    @property
    def amount_in_stable(self) -> float: return self.amount * self.price.price


In [ ]:
#| export

def symbol(token0: Token, token1: Token, pool_type: int) -> str:
    return f"CL{pool_type}-{token0.symbol}/{token1.symbol}" if pool_type > 0  else f"{'s' if pool_type == 0 else 'v'}AMM-{token0.symbol}/{token1.symbol}"

@dataclass(frozen=True)
class LiquidityPoolForSwap:
    """A more compact pool representation used for swaps"""

    lp: str
    type: int
    token0_address: str
    token1_address: str
    is_stable: bool
    # concentrated liquidity pools
    is_cl: bool

    @classmethod
    def from_tuple(cls, t: Tuple) -> "LiquidityPoolForSwap":
        token0, token1, pool_type = normalize_address(t[2]), normalize_address(t[3]), t[1]
        return LiquidityPoolForSwap(
            lp=normalize_address(t[0]),
            type=pool_type,
            token0_address=token0,
            token1_address=token1,
            is_stable=pool_type == 0,
            is_cl=pool_type > 0
        )

        


@dataclass(frozen=True)
class LiquidityPool:
    """Data class for Liquidity Pool

    based on:
    https://github.com/velodrome-finance/sugar/blob/v2/contracts/LpSugar.vy#L31
    """

    lp: str
    factory: str
    symbol: str
    type: int
    is_stable: bool
    # concentrated liquidity pools
    is_cl: bool
    total_supply: float
    decimals: int
    token0: Token
    reserve0: Amount
    token1: Token
    reserve1: Amount
    token0_fees: Amount
    token1_fees: Amount
    pool_fee: float
    gauge_total_supply: float
    emissions: Amount
    emissions_token: Token
    weekly_emissions: Amount
    nfpm: str
    alm: str

    @property
    def tvl(self) -> float: return self.reserve0.amount_in_stable + self.reserve1.amount_in_stable

    @property
    def total_fees(self) -> float:
        result = 0

        if self.token0_fees: result += self.token0_fees.amount_in_stable
        if self.token1_fees: result += self.token1_fees.amount_in_stable

        return result

    @property
    def pool_fee_percentage(self) -> float: return self.pool_fee / 100

    @property
    def volume_pct(self) -> float: return 100 / self.pool_fee_percentage

    @property
    def volume(self) -> Amount:
        t0 = self.token0_fees.amount_in_stable if self.token0_fees else 0
        t1 = self.token1_fees.amount_in_stable if self.token1_fees else 0
        return self.volume_pct * (t0 + t1)

    @property
    def token0_volume(self) -> float: return self.token0_fees.amount * self.volume_pct if self.token0_fees else 0

    @property
    def token1_volume(self) -> float: return self.token1_fees.amount * self.volume_pct if self.token1_fees else 0

    @property
    def apr(self) -> float:
        day_seconds, tvl = 24 * 60 * 60, self.tvl
        reward_value = self.emissions.amount_in_stable if self.emissions else 0
        reward = reward_value * day_seconds
        staked_pct = (
            100 * self.gauge_total_supply / self.total_supply if self.total_supply != 0 else 0
        )
        staked_tvl = tvl * staked_pct / 100
        return (reward / staked_tvl) * (100 * 365) if staked_tvl != 0 else 0

    @classmethod
    def from_tuple(
        cls, t: Tuple, tokens: Dict[str, Token], prices: Dict[str, Price]
    ) -> Optional["LiquidityPool"]:
        token0, token1, pool_type = normalize_address(t[7]), normalize_address(t[10]), t[4]
        token0_fees, token1_fees = t[23], t[24]
        emissions_token = normalize_address(t[20])
        emissions = t[19]

        seconds_in_a_week = 7 * 24 * 60 * 60

        # Sugar.all returns a tuple with the following structure:
        # { "name": "lp", "type": "address" },          <== 0
        # { "name": "symbol", "type": "string" },       <== 1
        # { "name": "decimals", "type": "uint8" },      <== 2
        # { "name": "liquidity", "type": "uint256" },   <== 3
        # { "name": "type", "type": "int24" },          <== 4
        # { "name": "tick", "type": "int24" },          <== 5
        # { "name": "sqrt_ratio", "type": "uint160" },  <== 6
        # { "name": "token0", "type": "address" },      <== 7
        # { "name": "reserve0", "type": "uint256" },    <== 8
        # { "name": "staked0", "type": "uint256" },     <== 9
        # { "name": "token1", "type": "address" },      <== 10
        # { "name": "reserve1", "type": "uint256" },    <== 11
        # { "name": "staked1", "type": "uint256" },     <== 12
        # { "name": "gauge", "type": "address" },        <== 13
        # { "name": "gauge_liquidity", "type": "uint256" },  <== 14
        # { "name": "gauge_alive", "type": "bool" },        <== 15
        # { "name": "fee", "type": "address" },             <== 16
        # { "name": "bribe", "type": "address" },           <== 17
        # { "name": "factory", "type": "address" },         <== 18
        # { "name": "emissions", "type": "uint256" },       <== 19
        # { "name": "emissions_token", "type": "address" },  <== 20
        # { "name": "pool_fee", "type": "uint256" },        <== 21
        # { "name": "unstaked_fee", "type": "uint256" },    <== 22
        # { "name": "token0_fees", "type": "uint256" },     <== 23
        # { "name": "token1_fees", "type": "uint256" },    <== 24
        # { "name": "nfpm", "type": "address" },           <== 25
        # { "name": "alm", "type": "address" }             <== 26

        token0, token1 = tokens.get(token0), tokens.get(token1)
        if not token0 or not token1: return None

        return LiquidityPool(
            lp=normalize_address(t[0]),
            factory=normalize_address(t[18]),
            symbol=symbol(token0, token1, pool_type),
            type=pool_type,
            # stable pools have type set to 0
            is_stable=pool_type == 0,
            is_cl=pool_type > 0,
            total_supply=t[3],
            decimals=t[2],
            token0=token0,
            reserve0=Amount.build(token0.token_address, t[8], tokens, prices),
            token1=token1,
            reserve1=Amount.build(token1.token_address, t[11], tokens, prices),
            token0_fees=Amount.build(token0.token_address, token0_fees, tokens, prices),
            token1_fees=Amount.build(token1.token_address, token1_fees, tokens, prices),
            pool_fee=t[21],
            gauge_total_supply=t[14],
            emissions_token=tokens.get(emissions_token),
            emissions=Amount.build(emissions_token, emissions, tokens, prices),
            weekly_emissions=Amount.build(emissions_token, emissions * seconds_in_a_week, tokens, prices),
            nfpm=normalize_address(t[25]),
            alm=normalize_address(t[26]),
        )

In [ ]:
#| export

@dataclass(frozen=True)
class LiquidityPoolEpoch:
    ts: int
    lp: str
    votes: int
    emissions: int
    incentives: List[Amount]
    fees: List[Amount]

    @classmethod
    def from_tuple(cls, t: Tuple, tokens: Dict[str, Token], prices: Dict[str, Price]) -> "LiquidityPoolEpoch":
        ts, lp, votes, emissions, incentives, fees = t[0], normalize_address(t[1]), t[2], t[3], t[4], t[5]
        def build_amount(token_address: str, amount: int) -> Amount:
            return Amount.build(normalize_address(token_address), amount, tokens, prices)

        return LiquidityPoolEpoch(ts=ts, lp=lp, votes=votes, emissions=emissions,
            incentives=[build_amount(i[0], i[1]) for i in incentives],
            fees=[build_amount(f[0], f[1]) for f in fees]
        )
# [
#     (1746662400, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 2056024776197125704049080, 19181568728350530, 
#         [('0x3417E54A51924C225330f8770514aD5560B9098D', 3000000000000000000000)],
#         [('0x3417E54A51924C225330f8770514aD5560B9098D', 267805653941732793034), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 81470986521137929477)]
#     ),
#     (1746057600, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 2056024832181766038591096, 19253831822417903, [('0x3417E54A51924C225330f8770514aD5560B9098D', 1200000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 44464184523591253285), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 64794494866571000870)]), (1745452800, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 2051385009634926652700680, 37306487407189160, [('0x3417E54A51924C225330f8770514aD5560B9098D', 1200000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 37571064173289335172), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 127265438536618692353)]), (1744848000, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 3977142636655506235648438, 37206144624961803, [('0x3417E54A51924C225330f8770514aD5560B9098D', 2600000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 92942471836846202824), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 299059396392390441882)]), (1744243200, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 3977088257079466249550992, 37116391390950033, [('0x3417E54A51924C225330f8770514aD5560B9098D', 2300000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 98379660134389607449), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 190148840949186514049)]), (1743638400, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 3975425402407602797294276, 37144347936477938, [('0x3417E54A51924C225330f8770514aD5560B9098D', 1500000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 81832664150680636426), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 106544187095976465204)]), (1743033600, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 3973767072055488422817753, 39111094487837607, [('0x3417E54A51924C225330f8770514aD5560B9098D', 3000000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 152980922867267765747), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 182333579676775942808)]), (1742428800, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 4255170361768291929377472, 39640502132165337, [('0x3417E54A51924C225330f8770514aD5560B9098D', 4300000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 201382309874805880678), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 376906113358557156648)]), (1741824000, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 4266102287179467714360647, 39985595681011872, [('0x3417E54A51924C225330f8770514aD5560B9098D', 5400000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 335197067444939975739), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 340822763729420176579)]), (1741219200, '0x7A7f1187c4710010DB17d0a9ad3fcE85e6ecD90a', 4257456360408014926013322, 40598073162727141, [('0x3417E54A51924C225330f8770514aD5560B9098D', 3600000000000000000000)], [('0x3417E54A51924C225330f8770514aD5560B9098D', 273437948081275951977), ('0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', 140346506674975968991)])
# ]

# ts: bigint;
# lp: `0x${string}`;
# votes: bigint;
# emissions: bigint;
# bribes: readonly {
#     token: `0x${string}`;
#     amount: bigint;
# }[];
# fees: readonly {
#     token: `0x${string}`;
#     amount: bigint;
# }[];    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()